In [1]:
'''
ssg convert process용 임시
-ep download
-convert to feed
-feed upload

*대용량 카탈로그의 경우
-피드 제품수 제한 (900만건 이상에서 오류남..) 
-facebook api 업로드 방식은 truncate insert / update only 두가지
-피드 나누어 제품 업로드해야함
-증분 업데이트시 개별 피드에 모두 해줘야함

*일단구현
-feed write시 
-카탈로그에 나눠진 피드 upload
-증분 업데이트 추가해야함
-증분 업데이트는 피드별로 모두 upload(update_only)

*ssg의 경우
-카탈로그1 -> 카탈로그2 수정만해서 재배포 (링크값)
-1,2 카탈로그 증분 업데이트
-머천센터 업로드는 500만건으로 고정되어있음(확인?)
-custom 링크/제외카테고리/이미지사이즈
'''

'\nssg convert process용 임시\n-ep download\n-convert to feed\n-feed upload\n\n*대용량 카탈로그의 경우\n-피드 제품수 제한 (900만건 이상에서 오류남..) \n-facebook api 업로드 방식은 truncate insert / update only 두가지\n-피드 나누어 제품 업로드해야함\n-증분 업데이트시 개별 피드에 모두 해줘야함\n\n*일단구현\n-feed write시 \n-카탈로그에 나눠진 피드 upload\n-증분 업데이트 추가해야함\n-증분 업데이트는 피드별로 모두 upload(update_only)\n\n*ssg의 경우\n-카탈로그1 -> 카탈로그2 수정만해서 재배포 (링크값)\n-1,2 카탈로그 증분 업데이트\n-머천센터 업로드는 500만건으로 고정되어있음(확인?)\n-custom 링크/제외카테고리/이미지사이즈\n'

## Import

In [11]:
import pandas as pd
import os
import gc
import zipfile
import requests
import asyncio, aiohttp, aiofiles
from starlette.config import Config
from urllib import parse
import json, time, datetime

## config

In [2]:
# ### ssg_eppe
config = {    
    "info" : {
        "media" : "facebook",
        "catalog_id" : "268046537186348",        
        "feed_id" : ["2499714026735797","457448645680512","887702468681795","933601424073448","433956697833525"],
        "name" : "ssg_ep_segment_test"
    },
    "ep" : {
        "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpAll.csv",
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348.csv",
        "format" : "csv",
        "sep" : ",",
        "encoding" : "utf-16",
        "cron" : ""
    },
    "ep_update" : {
        "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpBrief.csv",
        "path" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348_update.csv",
        "format" : "csv",
        "sep" : ",",
        "encoding" : "utf-16",
        "cron" : ""
    },
    "feed" : {
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_268046537186348.tsv",
    },
    "feed_update" : {
        "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_268046537186348_update.tsv"
    },
    "columns" : {
        "id" : "id",
        "title" : "title",
        "link" : "link",
        "image_link" : "image_link",
        "price" : "value",
        "brand" : "brand",
        "description" : "description",
        "availability" : "availability",
        "condition" : "condition"
    },
    "status" : ""
}

In [103]:
### ssg_eppe_ios
# update는 config새로 만들고 status값 추가
# config = {    
#     "info" : {
#         "media" : "facebook",
#         "catalog_id" : "225456985373646",        
#         "feed_id" : ["2433331320101804","704765896857210","136652408333414","250350879959740","2477650072531024"],
#         "name" : "ssg_ep_ios_segment_test"
#     },
#     "ep" : {
#         "url" : "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/test/data/ssg_facebookNoCkwhereEpAll.csv",
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_268046537186348.csv",        
#         "format" : "csv",
#         "sep" : ",",
#         "encoding" : "utf-16"
#     },    
#     "feed" : {
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_225456985373646.tsv",        
#     },
#     "columns" : {
#         "id" : "id",
#         "title" : "title",
#         "link" : "link",
#         "image_link" : "image_link",
#         "price" : "value",
#         "brand" : "brand",
#         "description" : "description",
#         "availability" : "availability",
#         "condition" : "condition"
#     }    
# }

In [104]:
# config = {    
#     "info" : {
#         "media" : "facebook",        
#         "catalog_id" : "432089861336082", # test_선상협
#         "feed_id" : ["236164118048821","254210579552181"],
#         "name" : "test_feed_upload"
#     },
#     "ep" : {        
#         "url" : "http://ep.hellonature.co.kr/PGD/EP/NAVER/all.txt",
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/ep/ep_432089861336082.csv",
#         "format" : "tsv",
#         "sep" : "\t",
#         "encoding" : "utf-8"
#     },
#     "feed" : {        
#         "path": "C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_432089861336082.tsv",
#     },
#     "columns" : {
#          "id" : "id",
#         "title" : "title",
#         "link" : "link",
#         "image_link" : "image_link",
#         "price_pc" : "value",
#         "brand" : "brand"
#     }    
# }

## Util

In [3]:
'''
다운로드시 파일 작성시간 확인 체크 필요
'''
async def epDownload(fromUrl, toPath):
    async with aiohttp.ClientSession() as session:
        async with session.get(fromUrl) as response:
            chunk_size = 1024*1024*10 # 10MB
            async with aiofiles.open(toPath, 'wb') as f:
                while True:
                    chunk = await response.content.read(chunk_size)                        
                    if not chunk : break
                    await f.write(chunk)

                # result = self.getInfo(toPath)
                # self.logger.info('Download complete '+str(result))
                # return result

In [4]:
def epLoad():
    # 원본 컬럼리스트 확인
    columns = pd.read_csv(
        config['ep']['path'], 
        nrows=1, sep=config['ep']['sep'], 
        encoding=config['ep']['encoding']
        )
    columns = list(columns)

    result = pd.read_csv(
        config['ep']['path'],
        nrows=None,
        chunksize=100000,
        header=0,
        dtype=str,
        sep=config['ep']['sep'],
        error_bad_lines=False,
        usecols=columns,
        encoding=config['ep']['encoding']
        )
    return result


In [5]:
async def feedWrite(num, feedPath, df):
    # print(feedPath)
    if num == 0:
        mode='w' # 새로쓰기
        header=True 
    else:
        mode='a' # 이어쓰기
        header=False
        
    df.to_csv(feedPath, 
                index=False, # 자체 인덱스제거
                sep='\t', 
                mode=mode,
                header=header, # 컬럼명 
                encoding='utf-8')

In [6]:
'''
feed파일 로컬에선 테스트불가
'''
async def facebookUpload( feed_id, feed_url, isUpdateOnly='false'):
    access_token = 'EAACpvLy2z30BAKxprnqZBpE8pfa08FVn6ya32d9TQk6wbXC4fmINmN8MVBur2eKGtYysSmMXMkttVg6s1bWu2yscUwEEZAdjJpRXyjxThbcuRYcKeSPMQm8ZAWhp4l5d6urUfxcjcY0UEVf46TpfhdlmSdKvlKbmxTIm6sc3xn2kIIWU6VX'
    api_url = f'https://graph.facebook.com/v9.0/{feed_id}/uploads'
    params ={'update_only': isUpdateOnly ,'access_token': access_token, 'url': feed_url}
    async with aiohttp.ClientSession() as session:
        async with session.post(api_url, data=params) as response:
            result = await response.text()            
            return result 

In [7]:
def convertFilter(dataFrame):
    result = dataFrame

    keys = list(config['columns'].keys())
    result = result[keys]
    result.rename(columns=config['columns'], inplace=False)
    
    # 공백제거
    for key in keys:
        result[key] = result[key].str.strip()

    # 기본값
    if 'availability' not in result.columns :
        result['availability'] = 'in stock'
    if 'condition' not in result.columns : 
        result['condition'] = 'new'
    if 'description' not in result.columns :
        result['description'] = result['title'].str.lower()
    else : 
        result['description'] = result['description'].str.lower()

    ########################
    ## custom filter 작성 ##
    ########################
    catalog_id = config['info']['catalog_id']


    # SSG_EPPE / SSG_EPPE_IOS
    if catalog_id == '268046537186348' or catalog_id == '225456985373646':                
        # 제외 title
        stopword = ['마스크','소독제','손소독겔','새니타이저','손살균제','snitizers','disinfection wipes']
        result = result[~result['title'].str.contains('|'.join(stopword))]

        # 이미지사이즈 조정?
        result['image_link'] = result['image_link'].str.replace('_500','_600', regex=False)

        # link값 변경        
        if catalog_id=='268046537186348' : # ssg_eppe            
            result['link'] = 'https://ad.adpool.co.kr/app/ssg/item/'+result['id'] # f''으로 적용하면 axis가 안맞는듯?

        elif catalog_id=='225456985373646' : # ssg_eppe_ios            
            result['link'] = 'http://m.ssg.com/item/itemView.ssg?itemId='+result['id']+'&gateYn=Y&mobilAppSvcNo=3'
                        

    elif catalog_id == '':
        pass

    return result

In [8]:
def zipped(fromPath, toPath):
    os.makedirs(os.path.dirname(toPath), exist_ok=True) # 경로확인/생성    
    zip = zipfile.ZipFile(toPath, 'w')
    zip.write(fromPath, compress_type=zipfile.ZIP_DEFLATED)    

## Convert Process

In [9]:

'''
대용량파일의 경우 분할하여 저장..(카탈로그 내 피드가 여러개인 경우)
'''
async def execute():
    # ep download
    # await epDownload(config['ep']['url'], config['ep']['path'])

    # total
    totalCount = 0
    totalChunkCount = 0
    for num, chunkDF in enumerate(epLoad()):
        totalCount = totalCount + chunkDF.shape[0]
        totalChunkCount = num # 최종 num 저장됨
        
        del[[chunkDF]]
        gc.collect()

    # segmentation
    feedCount = len(config['info']['feed_id']) # 카탈로그당 나눈 피드갯수
    chunkCountPerFeed = int(round(totalChunkCount/feedCount)) # 피드하나당 chunk갯수
    print(f'totalChunkCount : {totalChunkCount}')
    print(f'feedCount : {feedCount}')
    print(f'chunkCountPerFeed : {chunkCountPerFeed}')


    # chunk read
    count = 0
    feedIndex=0
    for num, chunkDF in enumerate(epLoad()):
        # print(chunkDF.head())
        count = count + chunkDF.shape[0]
        print(f'{round(num/totalChunkCount*100)}%', end='...')

        # filter
        chunkDF = convertFilter(chunkDF)

        # write        
        segmentIndex = num % chunkCountPerFeed # feed_id당 배분될 chunk만큼의 index 생성
        # print(f'segmentIndex : {segmentIndex}')
        
        if segmentIndex == 0: # 0일때마다 feed 분할
            feed_id = config['info']['feed_id'][feedIndex]
            feedSecPath = str(config['feed']['path']).replace('.tsv', f'_{feed_id}.tsv')
            if feedIndex < len(config['info']['feed_id'])-1:
                feedIndex = feedIndex + 1
    
            
        await feedWrite(segmentIndex, feedSecPath, chunkDF) #feed분할 적용       
        # await feedWrite(num, config['feed']['path'], chunkDF)
            
        del[[chunkDF]]
        gc.collect()

        # break

    # 속도가 아쉬움..압축을 안한다면?
    # catalog내 feed분할 업로드 위한 압축
    catalog_id = config['info']['catalog_id']
    for num, feed_id in enumerate(config['info']['feed_id']):
        feedPath = f'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_{catalog_id}_{feed_id}.tsv'
        zipPath = f'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/feed_{catalog_id}_{feed_id}.zip'
        zipped(feedPath, zipPath)

        # facebook api upload
        # print(await uploadAPI(feed_id, feedZipPath))

In [12]:
await execute()

totalChunkCount : 111
feedCount : 5
chunkCountPerFeed : 22
0%...1%...2%...3%...4%...5%...5%...6%...7%...8%...9%...10%...11%...12%...13%...14%...14%...15%...16%...17%...18%...19%...20%...21%...22%...23%...23%...24%...25%...26%...27%...28%...29%...30%...31%...32%...32%...33%...34%...35%...36%...37%...38%...39%...40%...41%...41%...42%...43%...44%...45%...46%...47%...48%...49%...50%...50%...51%...52%...53%...54%...55%...56%...57%...58%...59%...59%...60%...61%...62%...63%...64%...65%...66%...67%...68%...68%...69%...70%...71%...72%...73%...74%...75%...76%...77%...77%...78%...79%...80%...81%...82%...83%...84%...85%...86%...86%...87%...88%...89%...90%...91%...92%...93%...94%...95%...95%...96%...97%...98%...99%...100%...

In [62]:
await epDownload(config['ep']['url'], config['ep']['path'])

CancelledError: 

In [37]:
0%0

ZeroDivisionError: integer division or modulo by zero

In [38]:
0/1

0.0

In [41]:
# 중복제거 test
# 중복제거 test
tempPath = 'C:/Users/shsun/Documents/workspace/project/p1/f1_feed_change_min/data/feed/225456985373646/feed_225456985373646_temp.tsv'
tempLoad = pd.read_csv(tempPath,
            header=0, # header row                            
            dtype=str, # string type 인식
            sep='\t', # 명시
            # lineterminator='\r',
            error_bad_lines=False, # error skip
            encoding='utf-8')


In [42]:
print(tempLoad.shape[0])

7317167

In [44]:
tempLoad_1 = tempLoad.drop_duplicates(['id'], keep='first')
print(tempLoad_1.shape[0])

6398343


In [90]:
testDict = {"id1":{"path":"value1"}, "id2":{"path":"value2"}}
print(len(testDict.keys()))    
for value in testDict.keys() :
    print(testDict[value]['path'])
    

2
value1
value2
